# Create tables and load business register

Load libraries

In [1]:
import pandas as pd
import sqlalchemy

Download business register

In [13]:
er = pd.read_csv('http://hotell.difi.no/download/brreg/enhetsregisteret?download', sep=";",
                dtype={'orgnr': str, 'forradrpostnr': str, 'forradrkommnr': str, 
                       'ppostnr': str, 'nkode1': str, 'nkode2': str, 'nkode3': str,
                       'sektorkode': str, 'regnskap': float, 'hovedenhet': str, 'ansatte_antall': float})

In [22]:
er = pd.read_csv('enhetsregisteret?download', sep=";",
                dtype={'orgnr': str, 'forradrpostnr': str, 'forradrkommnr': str, 
                       'ppostnr': str, 'nkode1': str, 'nkode2': str, 'nkode3': str,
                       'sektorkode': str, 'regnskap': float, 'hovedenhet': str, 'ansatte_antall': float},
                parse_dates=['ansatte_dato', 'stiftelsesdato'])


Keep only businesses with employees

In [29]:
er_emp = er[(er['ansatte_antall'].isnull() == False)]
print(len(er_emp))

180187


Load NACE code list and slice/dice result before merge with business register

In [35]:
! curl 'http://data.ssb.no/api/klass/v1/classifications/6/codesAt?date=2015-01-01&language=en' -H 'Accept: text/csv; charset=UTF-8' > nace.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  171k    0  171k    0     0   474k      0 --:--:-- --:--:-- --:--:--  475k


In [36]:
import pandas as pd
nace = pd.read_csv('nace.csv', dtype=str)
toplevel = nace[(nace['level']=='1')][['code', 'shortName']]
codelevel = nace[(nace['level']=='2')][['code', 'parentCode']]
toplevel.rename(columns={'code': 'parentCode'}, inplace=True)
codes = codelevel.merge(toplevel)
codes.rename(columns={'code': 'nace_code'}, inplace=True)

In [32]:
er_emp['nace_code'] = er_emp['nkode1'].str[0:2]

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
er_nace = er_emp.merge(codes)

Connect to MySQL and create table vacancies.

In [43]:
%run ../connect_mysql.py

In [10]:
r = engine.execute("""CREATE TABLE IF NOT EXISTS `vacancies` (
  `id` int(11) unsigned NOT NULL AUTO_INCREMENT,
  `num_vacancies` int(11) DEFAULT NULL,
  `valid_from` date DEFAULT NULL,
  `valid_to` date DEFAULT NULL,
  `isco08` varchar(4) DEFAULT NULL,
  `orgnr` varchar(9) DEFAULT NULL,
  `text` varchar(8000) DEFAULT NULL,
  `title` varchar(500) DEFAULT NULL,
  PRIMARY KEY (`id`),
  KEY `orgnr_ix` (`orgnr`),
  KEY `isco08_ix` (`isco08`)
) ENGINE=InnoDB AUTO_INCREMENT=1188 DEFAULT CHARSET=utf8mb4;""")

In [41]:
try:
    er_nace.to_sql('er', engine, if_exists='replace')
except:
    print("failed")

In [46]:
%run ../connect_mysql.py
r = engine.execute("""CREATE INDEX orgnr_ix ON er (orgnr)""")